# Include libraries & configure cells

In [1]:
# Libraries
%matplotlib widget
#%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as mpe
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib
import sys
from scipy import stats
import math

In [2]:
# Full width cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Load data and clean it

## Read .csv & rename columns

In [3]:
# Load our data from file
LocationAllData = "../data-cleanup/merged-cleaned-csv/driving_data_merged_1_to_30.csv"
df = pd.read_csv(LocationAllData)

# Give the columns names in the same style and give some entries clearer names
df = df.rename(columns={
    "Attempt nr":"attempt", "userID": "user_id", "evisID":"evis_id", "timeStamp":"timestamp", "currentStateOfCharge":"current_soc", "energyConsumed":"energy_consumed", "energyUsage":"power_usage",
    "guesstimatedDistanceLeft":"range_estimate","distanceTraveled":"distance_traveled", "throttlePosition":"throttle_position", "breakPosition":"break_position", "steeringWheelRot":"wheel_rot", "yPosition":"road_height"
})

## Assign *distance window*
Add a new column with a so called "distance_window" this will be used to group data points in order to calculate average over distance

In [4]:
# Decide which distance interval to average over
distanceWindowWidth = 50

def assignDistanceWindow(distance):
    distanceWindowIndex = round(distance/distanceWindowWidth)
    return distanceWindowWidth * distanceWindowIndex

# Add a new row which containts which distance group each row belongs to
df["distance_window"] = df["distance_traveled"].apply(assignDistanceWindow)

# The range estimate seen by the driver using the guess-o-meter is truncated, i.e only the integer is seen
# 1.0 - 1.9 -> 1.0
df["seen_range_estimate"] = df["range_estimate"].apply(lambda x: int(x))

## Exclusion of participants

Here we should exclude participants who did not execute the task properly, e.g not using the dashboard at all or misunderstanding the task  
List of participants I think should be excluded with a reason attached

* id:5 - Guess - Participant didn't use the Range Estimate number in the dashboard  
* id:10 - Diff - Participant didn't use bars at all, only focused on the speed and energy usage  
* id:16 - Diff - Didn't understand the blue line  
* id:25 - Guess - Seemed like the participant didn't understand that going at a lower speed was allowed
* id:29 - Guess - Didn't want to drive too slow on the highway, mentioned that it's illegal to go < 100 km/h

Not sure about these
* id:14 - Diff - Experimented quite a bit but managed to complete the drive at 2nd attempt
* id:15 - Guess - Tried to regain energy by breaking quite a bit

In [5]:
# Id of participants we want to exclude from the study
exclusion_list = [5, 10, 14, 16, 25, 29]
exclusion_list = [5, 10, 16, 25, 29]
#exclusion_list = []
# Remove all rows which are of participants who are in the exclusion_list
df_excluded = df[~df.user_id.isin(exclusion_list)]

# Get all the unique user id's
all_user_ids = df_excluded.user_id.unique()
# Total number of users
number_of_users = all_user_ids.size
individual_attempts_are_hidden = False

# Calculation of averages and STD

## Calculate mean over distance for all attempts separately
I.e a new data frame is created where each attempt performed by a single participant is averaged over each distance window.  
Resulting in fewer rows but with a column with fixed distances

`average_individual` contains each individual attempt over distance traveled averaged at each distance window

In [6]:
#
average_individual = df_excluded.groupby(["attempt", "evis_id", "user_id", "distance_window"]).mean().reset_index()
#average_individual

## Calculate mean and STD of desired variables for the 4 different groups
These are the four defined groups
1. Diff + COPE1 - Attempt #1
2. Diff + COPE1 - Attempt #2
3. Guess-o-meter - Attempt #1
4. Guess-o-meter - Attempt #2

`average_group_based` contain the average variables of each distance window for each of the four groups


In [7]:
average_individual

,attempt,evis_id,user_id,distance_window,timestamp,current_soc,energy_consumed,power_usage,range_estimate,speed,distance_traveled,throttle_position,break_position,wheel_rot,xPosition,road_height,zPosition,seen_range_estimate
0,1,DiffAndCOPE1,2,0,7.918182,1.196429,0.005132,2.723160,-1.000000,5.728570,5.444120,0.311293,0.0,-0.017354,-13.761029,0.088220,488.207819,-1.0
1,1,DiffAndCOPE1,2,50,19.334333,1.177502,0.024057,9.826560,-1.000000,25.905023,48.208120,0.581217,0.0,0.003077,26.334722,0.086431,503.798464,-1.0
2,1,DiffAndCOPE1,2,100,25.547711,1.162704,0.038856,7.501454,-1.000000,33.209393,99.576555,0.578970,0.0,0.004771,74.658660,0.085698,521.734050,-1.0
3,1,DiffAndCOPE1,2,150,30.739478,1.153444,0.048116,5.559260,-1.000000,35.921378,149.706561,0.565676,0.0,0.005956,122.193344,0.085423,537.656969,-1.0
4,1,DiffAndCOPE1,2,200,35.395645,1.140608,0.060952,14.079423,-1.000000,41.635776,198.894921,0.674357,0.0,0.015305,169.723906,0.084865,550.661632,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7287,2,GuessOMeter,27,7800,773.003012,0.205619,1.011195,1.193401,1.703102,42.625795,7799.766113,0.558596,0.0,-0.004826,488.553287,-4.016750,-3846.038976,1.0
7288,2,GuessOMeter,27,7850,777.240126,0.203949,1.012865,1.642473,1.692213,42.322964,7849.827810,0.558596,0.0,-0.006873,460.519112,-4.587503,-3887.423406,1.0
7289,2,GuessOMeter,27,7900,781.429209,0.199564,1.017250,8.550583,1.656894,44.019485,7899.461340,0.659282,0.0,0.000236,434.524329,-5.034237,-3929.636203,1.0
7290,2,GuessOMeter,27,7950,785.286467,0.185902,1.030912,13.607345,1.538842,49.961410,7949.715632,0.698765,0.0,-0.004779,408.716695,-5.362894,-3972.842690,1.0


In [8]:
#
average_groups = average_individual.groupby(["attempt", "evis_id", "distance_window"], as_index=False)

average_group_based = average_groups[["speed", "current_soc", "road_height", "energy_consumed", "range_estimate", "power_usage"]].agg(
    {"speed": ["mean", "std"], 
     "current_soc": ["mean", "std"], 
     "energy_consumed": ["mean", "std"],
     "range_estimate": ["mean", "std"],
     "power_usage": ["mean", "std"],
     "road_height": "mean"
    })

#average_group_based

## Average road height
Calculate the average road height at each `distance_window`

In [9]:
# Calculate the average road height over all the drives
# We can ignore the warning here, it's still fast
average_road_height = average_group_based[["distance_window","road_height"]].groupby(["distance_window"], as_index=False).mean()

/home/steve/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


# Number of successful attempts vs failed ones


## Number of valid participants

In [10]:
#
total_number_of_participants = len(df_excluded["user_id"].unique())

unique_ids = df_excluded.groupby(["evis_id", "user_id"], as_index=False).count()
number_of_participants_diff = len(unique_ids[unique_ids["evis_id"] == "DiffAndCOPE1"])
number_of_participants_guess = len(unique_ids[unique_ids["evis_id"] == "GuessOMeter"])

pd.DataFrame(data={"Diff_COPE1":[number_of_participants_diff], "Guess_o_meter":[number_of_participants_guess]}, index=["nr_of_participants"])


,Diff_COPE1,Guess_o_meter
nr_of_participants,13,12


## Create a new data frame with the number of fails and successes in each group
`each_user_final_data_success` & `each_user_final_data_fail` contain the last averaged data point for each individual attempt  
`df_fail_success` is the resulting data frame with success and fail count

In [11]:
# Group based on attempt, evis and user id.
each_user = average_individual.groupby(["attempt", "evis_id","user_id"])

# Create a new df with the last datapoint for each attempt
each_user_final_data = pd.concat([each_user.tail(1)])

each_user_final_data_success = each_user_final_data[each_user_final_data["distance_window"] == 8000]
each_user_final_data_fail = each_user_final_data[each_user_final_data["distance_window"] != 8000]

df_fail_success = each_user_final_data_success.groupby(["attempt", "evis_id"])[["user_id"]].count().rename(columns={"user_id":"successes"})
fails = each_user_final_data_fail.groupby(["attempt", "evis_id"])[["user_id"]].count()

# Add column with fails onto our successes df
df_fail_success["fails"] = fails["user_id"]
df_fail_success = df_fail_success.fillna(0)

df_fail_success["success_rate"] = df_fail_success["successes"] / (df_fail_success["successes"] + df_fail_success["fails"])

df_fail_success

successes  fails  success_rate
attempt evis_id                                     
1       DiffAndCOPE1          8    5.0      0.615385
        GuessOMeter           3    9.0      0.250000
2       DiffAndCOPE1         13    0.0      1.000000
        GuessOMeter           9    3.0      0.750000

## Test statistical significance
Stats fisher test: If p-value is < 0.05 **H0** is rejected in favor of **H1**  
**H0**: Dashboard and success rate are independent (Dashboard does not affect success rate, equal chance of making it with either EVIS)  
**H1**: Dashboard and success rate are not independent (Dashboard affects success rate, it matters what EVIS you use)  

In [12]:
# Stats fisher test

a1 = df_fail_success.loc[1].T
#a1.loc["successes"].values

oddsratio, pvalue = stats.fisher_exact([a1.loc["successes"].values, a1.loc["fails"].values])
print("p-value first attempt")
print(pvalue)

a2 = df_fail_success.loc[2].T
#a2.loc["successes"].values

oddsratio, pvalue = stats.fisher_exact([a2.loc["successes"].values, a2.loc["fails"].values])
print("\np-value second attempt")
print(pvalue)

p-value first attempt
0.11070130569390259

p-value second attempt
0.0956521739130436


# Graph the attempts over distance traveled

## Default plot settings
Default font sizes and colors

In [13]:
# Plot window presets & color constants
plt.style.use("default")
params = {
    'font.size': 12,
    'axes.labelsize': 15,
    'axes.titlesize': 15,
    'legend.fontsize': 12,
    'xtick.labelsize': 15,
    'ytick.labelsize': 15
}
matplotlib.rcParams.update(params)

color_road = "k"
color_diff = "#FFA85C"
color_diff_border = "#E06900"
color_diff_a1 = "#1B998B"
color_guess = "#3F9CDE"
color_guess_a1 = "#1B998B"
color_guess_border = "#1B679D"

## SoC over distance
SoC over distance traveled plots

### Attempt #1

In [14]:
#
def didUserMakeIt(group):
    return group["distance_window"].iloc[-1] == 8000

# Class for the individual attempt line
class IndividualAttemptLine:
    line_highlight = {"a": 1.0, "w": 3.0}
    line_normal = {"a": 0.4, "w": 2.0}
    marker_highlight = 1.0
    marker_normal = 0.7
    
    def __init__(self, ax, name, group):
        self.group = group
        self.driver_id = name[2]
        self.draw_order_base = self.driver_id * 3
        self.is_highlighted = False
        self.color = color_guess if name[1] == "GuessOMeter" else color_diff
        self.color_strong = color_guess_border if name[1] == "GuessOMeter" else color_diff_border
        
        # Individual Attempt
        self.line = ax.plot(group["distance_window"], group["power_usage"], 
                            color=self.color, zorder=self.draw_order_base, 
                            linewidth=self.line_normal["w"], alpha=self.line_normal["a"])[0]
        
        # Marking for where the user ended up
        self.marker = ax.plot(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                              color=self.color, zorder=self.draw_order_base + 1, marker="o", markersize=25, 
                              alpha=self.marker_normal)[0]
        
        # Mark the endpoint with the user id as well
        self.text = ax.text(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                            s=self.driver_id, color="k", size="small", zorder=self.draw_order_base + 2, clip_on=True, 
                            horizontalalignment='center', verticalalignment='center')
        
        
    
    def highlight(self, value):
        self.is_highlighted = value
        
        self.line.set_alpha(self.line_highlight["a"] if self.is_highlighted else self.line_normal["a"])
        self.line.set_linewidth(self.line_highlight["w"] if self.is_highlighted else self.line_normal["w"])
        self.line.set_zorder(500 + self.driver_id * 3 + 1 if self.is_highlighted else self.driver_id * 3 + 1)
        
        # Use path effect to see overlaps more clearly
        pe1 = [
            mpe.Stroke(linewidth=5, foreground=self.color_strong),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.line.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.marker.set_alpha(self.marker_highlight if self.is_highlighted else self.marker_normal)
        self.marker.set_zorder(500 + self.driver_id * 3 + 2 if self.is_highlighted else self.driver_id * 3 + 2)
        self.marker.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.text.set_zorder((500 + self.driver_id * 3 + 3) if self.is_highlighted else (self.driver_id * 3 + 3))
        
        # If we highlight and the rest of the lines are hidden -> show it
        if self.is_highlighted:
            self.text.set_alpha(1)
        # If we remove highlight and the rest are hidden -> hide it
        if not self.is_highlighted and individual_attempts_are_hidden:
            self.setHide(True)
    
    def setHide(self, hide, force=False):
        # If the line is alreday highlighted don't hide it! Unless fore is set to true
        if self.is_highlighted and force == False:
            return
        
        self.line.set_alpha(0 if hide else self.line_normal["a"])
        self.marker.set_alpha(0 if hide else self.marker_normal)
        self.text.set_alpha(0 if hide else 1)

    def showIfFailed(self):
        if didUserMakeIt(self.group):
            self.setHide(True, True)
        else:
            self.setHide(False)
        
class MeanWithSTD:
    def __init__(self, ax, name, group):
        self.color = color_guess if name[1] == "GuessOMeter" else color_diff
        self.attempt = name[0]
        self.std_alpha = 0.5
        self.color_strong = color_guess_border if name[1] == "GuessOMeter" else color_diff_border
        
        self.average_line = ax.plot(group["distance_window"], group["power_usage"]["mean"],
            color=self.color, zorder=100, linewidth=2, alpha=1)[0]
        
        # Use path effect to see overlaps more clearly
        self.pe1 = [
            mpe.Stroke(linewidth=4, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.average_line.set_path_effects(self.pe1)
        
        self.zorder = 1 if name[1] == "GuessOMeter" else 2
        self.std_band = ax.fill_between(
            group["distance_window"],
            group["power_usage"]["mean"] + group["power_usage"]["std"],
            group["power_usage"]["mean"] - group["power_usage"]["std"],
            color=self.color, alpha=self.std_alpha, zorder=self.zorder)

    def setHide(self, hide):
        self.average_line.set_alpha(0 if hide else 1)
        self.std_band.set_alpha(0 if hide else self.std_alpha)
        self.average_line.set_path_effects([] if hide else self.pe1)

In [41]:
# Plot lines
whichAttempt = 1

# Individual attempts plot
output_1 = widgets.Output()

with output_1:
    fig_1, ax_1 = plt.subplots(figsize=(10, 6))
ax_1.set_xlabel("Distance [m]")
ax_1.set_ylabel("Power [kW]")
ax_1.set_title("Average power over distance: attempt #" + str(whichAttempt))
ax_1.grid(True)

### Draw the plot objects ###

# Plot the individual attempts as transparent lines. At the end of each attempt, plot a dot to mark it
driving_lines_obj_dict_1 = {}
average_individual_grouped = average_individual.groupby(["attempt","evis_id","user_id"])
for name,group in average_individual_grouped:
    if name[0] != whichAttempt: 
        continue
    
    individual_attempt = IndividualAttemptLine(ax_1, name, group)
    driving_lines_obj_dict_1[name[2]] = individual_attempt

# Plot the average line with +- standard deviation
average_group_based_grouped = average_group_based.groupby(["attempt","evis_id"])
mean_std_obj_dict_1 = {}
for name,group in average_group_based_grouped:
    if name[0] != whichAttempt: 
        continue
    
    mean_with_std = MeanWithSTD(ax_1, name, group)
    mean_std_obj_dict_1[name[1]] = mean_with_std       

sampled_distances = np.arange(0, 8000 + 1, distanceWindowWidth)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_diff = individual_pivot.round(1)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_guess = individual_pivot.round(1)

# IDEA: Check for statistical difference using ttest at each distance window.
# If ther's a difference, paint the background in a particular color to highlight that "here wer have a statistical difference!!!"
        
### Road height ###

# Plot road height
ax2_1 = ax_1.twinx()
ax2_1.set_ylim(-10, 30)
ax2_1.grid(True, linestyle="--", linewidth=2)
ax2_1.set_ylabel('Elevation [m]', color=color_road)
road_height_plot, = ax2_1.plot(average_road_height["distance_window"], average_road_height["road_height"], 
                        color=color_road, zorder=2, linewidth=2, linestyle="--", alpha=1, label="Elevation")

# Marker for where the 110 km/h signs are
vline_1 = ax_1.axvline(x=3100, color="k", linestyle="-", linewidth=2, label="110 km/h sign")

# Create a legend
first_patch = patches.Patch(color=color_guess, label="Guess-o-meter")
second_patch = patches.Patch(color=color_diff, label="Diff + COPE1")
ax_1.legend(handles=[first_patch, second_patch, road_height_plot, vline_1], loc="upper right").set_zorder(2000)

ax_1.set_yticks(np.arange(0,101,10))

# Prevent secondary axis to be drawn ontop of everything
ax_1.set_zorder(1)
ax_1.patch.set_visible(False)

In [42]:
### Widgets for updating the graph ###

def highlightLine(x):
    id = int(x.owner.description)
    driving_lines_obj_dict_1[id].highlight(x.new)

# Allow user to highlight individual attempts
checkboxes_guess = [widgets.Label("Guess-o-meter")]
checkboxes_copediff = [widgets.Label("COPE1 + diff")]
for i in range(number_of_users):
    checkbox = widgets.Checkbox(description=str(all_user_ids[i]), value=False, layout={"width":"50px"}, indent=False)
    checkbox.observe(highlightLine, "value")
    if all_user_ids[i]%2:
        checkboxes_guess.append(checkbox)
    else:
        checkboxes_copediff.append(checkbox)

container_guess_checkboxes = widgets.HBox(checkboxes_guess)
container_diffcope_checkboxes = widgets.HBox(checkboxes_copediff)
checkboxes = widgets.VBox([container_guess_checkboxes, container_diffcope_checkboxes])

# Allow user to update the range of each axis
def updateXRange(x):
    margin = 200
    ax_1.set_xlim(x.new[0] - margin, x.new[1] + margin)
def updateYRange(y):
    margin = 0
    ax_1.set_ylim(y.new[0] - margin, y.new[1] + margin)
def updateYRangeSecondary(y):
    margin = 1
    ax2_1.set_ylim(y.new[0] - margin, y.new[1] + margin)
    
def createRangeSlider(min, max, step, desc):
    return widgets.IntRangeSlider(
        value=[min,max],
        min=min,
        max=max,
        step=step,
        description=desc
    )

range_slider_x = createRangeSlider(0,8000, 100, "X Range")
range_slider_x.observe(updateXRange,"value")

range_slider_y = createRangeSlider(0,2,0.1, "Y Range (left)")
range_slider_y.observe(updateYRange,"value")

range_slider_y_secondary = createRangeSlider(-10,40,1, "Y Range (Right)")
range_slider_y_secondary.observe(updateYRangeSecondary,"value")

# Allow user to select what to show
def toggleIndividualAttempts(show):
    global individual_attempts_are_hidden # This is needed to change global variables inside a function
    individual_attempts_are_hidden = not show.new
    for line in driving_lines_obj_dict_1:
        driving_lines_obj_dict_1[line].setHide(individual_attempts_are_hidden)
            
show_individual_attempts_checkbox = widgets.Checkbox(description="Show individual attempts", value=True)
show_individual_attempts_checkbox.observe(toggleIndividualAttempts, "value")

# TODO: Show failed attempts
def showFailedAttempts(show):
    for line in driving_lines_obj_dict_1:
        driving_lines_obj_dict_1[line].showIfFailed()
    
show_failed_attempts = widgets.Button(description="Show failed attempts")
show_failed_attempts.on_click(showFailedAttempts)

def saveImage(save):
    fig_1.savefig("../figures-local/a1_speed_distance.png", dpi=300)

save_plot = widgets.Button(description="Save as image")
save_plot.on_click(saveImage)

# TODO: Show successful attempts

# TODO: Show average speed

def toggleMeanSTD(show):
    for plot_obj in mean_std_obj_dict_1:
        mean_std_obj_dict_1[plot_obj].setHide(not show.new)

toggle_mean_std = widgets.Checkbox(description="Show mean & std", value=True)
toggle_mean_std.observe(toggleMeanSTD, "value")

# TODO: Show standard deviation

box_2 = widgets.HBox([range_slider_x, range_slider_y, range_slider_y_secondary])
box_3 = widgets.HBox([show_individual_attempts_checkbox, toggle_mean_std, show_failed_attempts, save_plot])
box_1 = widgets.VBox([output_1, checkboxes])
display(widgets.VBox([box_1, box_2, box_3]))

### Attempt #2

In [32]:
# Plot lines
whichAttempt = 2

# Individual attempts plot
output = widgets.Output()

with output:
    fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel("Distance [m]")
ax.set_ylabel("Power [kW]")
ax.set_title("Average power over distance: attempt #" + str(whichAttempt))
ax.grid(True)

### Draw the plot objects ###

# Plot the individual attempts as transparent lines. At the end of each attempt, plot a dot to mark it
driving_lines_class_dict = {}
average_individual_grouped = average_individual.groupby(["attempt","evis_id","user_id"])
for name,group in average_individual_grouped:
    if name[0] != whichAttempt: 
        continue
    
    individual_attempt = IndividualAttemptLine(ax, name, group)
    driving_lines_class_dict[name[2]] = individual_attempt

# Plot the average line with +- standard deviation
average_group_based_grouped = average_group_based.groupby(["attempt","evis_id"])
mean_std_class_dict = {}
for name,group in average_group_based_grouped:
    if name[0] != whichAttempt: 
        continue
    
    mean_with_std = MeanWithSTD(ax, name, group)
    mean_std_class_dict[name[1]] = mean_with_std       

sampled_distances = np.arange(0, 8000 + 1, distanceWindowWidth)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_diff = individual_pivot.round(1)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_guess = individual_pivot.round(1)

# IDEA: Check for statistical difference using ttest at each distance window.
# If ther's a difference, paint the background in a particular color to highlight that "here wer have a statistical difference!!!"
        
### Road height ###

# Plot road height
ax2 = ax.twinx()
ax2.set_ylim(-10, 30)
ax2.grid(True, linestyle="--", linewidth=2)
ax2.set_ylabel('Elevation [m]', color=color_road)
road_height_plot, = ax2.plot(average_road_height["distance_window"], average_road_height["road_height"], 
                        color=color_road, zorder=2, linewidth=2, linestyle="--", alpha=1, label="Elevation")

# Marker for where the 110 km/h signs are
vline = ax.axvline(x=3100, color="k", linestyle="-", linewidth=2, label="110 km/h sign")

# Create a legend
first_patch = patches.Patch(color=color_guess, label="Guess-o-meter")
second_patch = patches.Patch(color=color_diff, label="Diff + COPE1")
ax.legend(handles=[first_patch, second_patch, road_height_plot, vline], loc="upper right").set_zorder(2000)

ax.set_yticks(np.arange(0,101,10))

# Prevent secondary axis to be drawn ontop of everything
ax.set_zorder(1)
ax.patch.set_visible(False)

In [33]:
### Widgets for updating the graph ###

def highlightLine(x):
    id = int(x.owner.description)
    driving_lines_class_dict[id].highlight(x.new)

# Allow user to highlight individual attempts
checkboxes_guess = [widgets.Label("Guess-o-meter:    ")]
checkboxes_copediff = [widgets.Label("Diff + COPE1:    ")]
for i in range(number_of_users):
    checkbox = widgets.Checkbox(description=str(all_user_ids[i]), value=False, layout={"width":"50px"}, indent=False)
    checkbox.observe(highlightLine, "value")
    if all_user_ids[i]%2:
        checkboxes_guess.append(checkbox)
    else:
        checkboxes_copediff.append(checkbox)

container_guess_checkboxes = widgets.HBox(checkboxes_guess)
container_diffcope_checkboxes = widgets.HBox(checkboxes_copediff)
checkboxes = widgets.VBox([container_guess_checkboxes, container_diffcope_checkboxes])

# Allow user to update the range of each axis
def updateXRange(x):
    margin = 200
    ax.set_xlim(x.new[0] - margin, x.new[1] + margin)
def updateYRange(y):
    margin = 0
    ax.set_ylim(y.new[0] - margin, y.new[1] + margin)
def updateYRangeSecondary(y):
    margin = 1
    ax2.set_ylim(y.new[0] - margin, y.new[1] + margin)
    
def createRangeSlider(min, max, step, desc):
    return widgets.IntRangeSlider(
        value=[min,max],
        min=min,
        max=max,
        step=step,
        description=desc
    )

range_slider_x = createRangeSlider(0,8000, 100, "X Range")
range_slider_x.observe(updateXRange,"value")

range_slider_y = createRangeSlider(0,140,10, "Y Range (left)")
range_slider_y.observe(updateYRange,"value")

range_slider_y_secondary = createRangeSlider(-10,40,1, "Y Range (Right)")
range_slider_y_secondary.observe(updateYRangeSecondary,"value")

# Allow user to select what to show
def toggleIndividualAttempts(show):
    global individual_attempts_are_hidden # This is needed to change global variables inside a function
    individual_attempts_are_hidden = not show.new
    for line in driving_lines_class_dict:
        driving_lines_class_dict[line].setHide(individual_attempts_are_hidden)
            
show_individual_attempts_checkbox = widgets.Checkbox(description="Show individual attempts", value=True)
show_individual_attempts_checkbox.observe(toggleIndividualAttempts, "value")

# TODO: Show failed attempts
def showFailedAttempts(show):
    for line in driving_lines_class_dict:
        driving_lines_class_dict[line].showIfFailed()
    
show_failed_attempts = widgets.Button(description="Show failed attempts")
show_failed_attempts.on_click(showFailedAttempts)

def saveImage(save):
    fig.savefig("../figures-local/a2_speed_distance.png", dpi=300)

save_plot = widgets.Button(description="Save as image")
save_plot.on_click(saveImage)

# TODO: Show successful attempts

# TODO: Show average speed

def toggleMeanSTD(show):
    for plot_obj in mean_std_class_dict:
        mean_std_class_dict[plot_obj].setHide(not show.new)

toggle_mean_std = widgets.Checkbox(description="Show mean & std", value=True)
toggle_mean_std.observe(toggleMeanSTD, "value")

box_2 = widgets.HBox([range_slider_x, range_slider_y, range_slider_y_secondary])
box_3 = widgets.HBox([show_individual_attempts_checkbox, toggle_mean_std, show_failed_attempts, save_plot])
box_1 = widgets.VBox([output, checkboxes])

display(widgets.VBox([box_1, box_2, box_3]))

### Attempt 1 & 2 - Diff + COPE1

In [38]:
#
# Class for the individual attempt line, modified for showing both attempts
class IndividualAttemptLine2:
    line_highlight = {"a": 1.0, "w": 3.0}
    line_normal = {"a": 0.4, "w": 2.0}
    marker_highlight = 1.0
    marker_normal = 0.7
    
    def __init__(self, ax, name, group):
        self.group = group
        self.driver_id = name[2]
        self.draw_order_base = self.driver_id * 3
        self.is_highlighted = False
        self.color = color_diff if name[0] == 2 else color_diff_a1
        self.color_strong = color_guess_border if name[0] == 1 else color_diff_border
        
        # Individual Attempt
        self.line = ax.plot(group["distance_window"], group["power_usage"], 
                            color=self.color, zorder=self.draw_order_base, 
                            linewidth=self.line_normal["w"], alpha=self.line_normal["a"])[0]
        
        # Marking for where the user ended up
        self.marker = ax.plot(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                              color=self.color, zorder=self.draw_order_base + 1, marker="o", markersize=25, 
                              alpha=self.marker_normal)[0]
        
        # Mark the endpoint with the user id as well
        self.text = ax.text(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                            s=self.driver_id, color="k", size="small", zorder=self.draw_order_base + 2, clip_on=True, 
                            horizontalalignment='center', verticalalignment='center')
    
    def highlight(self, value):
        self.is_highlighted = value
        
        self.line.set_alpha(self.line_highlight["a"] if self.is_highlighted else self.line_normal["a"])
        self.line.set_linewidth(self.line_highlight["w"] if self.is_highlighted else self.line_normal["w"])
        self.line.set_zorder(500 + self.driver_id * 3 + 1 if self.is_highlighted else self.driver_id * 3 + 1)
        
        # Use path effect to see overlaps more clearly
        pe1 = [
            mpe.Stroke(linewidth=5, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.line.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.marker.set_alpha(self.marker_highlight if self.is_highlighted else self.marker_normal)
        self.marker.set_zorder(500 + self.driver_id * 3 + 2 if self.is_highlighted else self.driver_id * 3 + 2)
        self.marker.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.text.set_zorder((500 + self.driver_id * 3 + 3) if self.is_highlighted else (self.driver_id * 3 + 3))
        
        # If we highlight and the rest of the lines are hidden -> show it
        if self.is_highlighted:
            self.text.set_alpha(1)
        # If we remove highlight and the rest are hidden -> hide it
        if not self.is_highlighted and individual_attempts_are_hidden:
            self.setHide(True)
    
    def setHide(self, hide, force=False):
        # If the line is alreday highlighted don't hide it! Unless fore is set to true
        if self.is_highlighted and force == False:
            return
        
        self.line.set_alpha(0 if hide else self.line_normal["a"])
        self.marker.set_alpha(0 if hide else self.marker_normal)
        self.text.set_alpha(0 if hide else 1)

    def showIfFailed(self):
        if didUserMakeIt(self.group):
            self.setHide(True, True)
        else:
            self.setHide(False)
        
class MeanWithSTD2:
    def __init__(self, ax, name, group):
        self.color = color_diff if name[0] == 2 else color_diff_a1
        self.attempt = name[0]
        self.std_alpha = 0.4
        self.color_strong = color_guess_border if name[1] == "GuessOMeter" else color_diff_border
        
        self.average_line = ax.plot(group["distance_window"], group["power_usage"]["mean"],
            color=self.color, zorder=100, linewidth=2, alpha=1)[0]
        
        # Use path effect to see overlaps more clearly
        self.pe1 = [
            mpe.Stroke(linewidth=4, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.average_line.set_path_effects(self.pe1)
        
        self.zorder = 1 if name[1] == "GuessOMeter" else 2
        self.std_band = ax.fill_between(
            group["distance_window"],
            group["power_usage"]["mean"] + group["power_usage"]["std"],
            group["power_usage"]["mean"] - group["power_usage"]["std"],
            color=self.color, alpha=self.std_alpha, zorder=self.zorder)

    def setHide(self, hide):
        self.average_line.set_alpha(0 if hide else 1)
        self.std_band.set_alpha(0 if hide else self.std_alpha)
        self.average_line.set_path_effects([] if hide else self.pe1)

In [39]:
# Plot lines
whichAttempt = 2

# Individual attempts plot
output = widgets.Output()

with output:
    fig_3, ax_3 = plt.subplots(figsize=(10, 6))
ax_3.set_xlabel("Distance [m]")
ax_3.set_ylabel("Power [kw]")
ax_3.set_title("Average power over distance traveled: attempt 1 & 2 diff+COPE1")
ax_3.grid(True)

### Draw the plot objects ###

# Plot the individual attempts as transparent lines. At the end of each attempt, plot a dot to mark it
# This should be separated by which attempt it was!
driving_lines_obj_dict_3 = {}
average_individual_grouped = average_individual.groupby(["attempt","evis_id","user_id"])
for name,group in average_individual_grouped:
    if name[1] != "DiffAndCOPE1": 
        continue
    individual_attempt = IndividualAttemptLine2(ax_3, name, group)
    # Let the key be id+attempt
    id = str(name[2]) + str(name[0])
    driving_lines_obj_dict_3[id] = individual_attempt

# Plot the average line with +- standard deviation
average_group_based_grouped = average_group_based.groupby(["attempt","evis_id"])
mean_std_obj_dict_3 = {}
for name,group in average_group_based_grouped:
    if name[1] != "DiffAndCOPE1": 
        continue
    
    mean_with_std = MeanWithSTD2(ax_3, name, group)
    mean_std_obj_dict_3[name[0]] = mean_with_std       

sampled_distances = np.arange(0, 8000 + 1, distanceWindowWidth)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_diff = individual_pivot.round(1)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_guess = individual_pivot.round(1)

# IDEA: Check for statistical difference using ttest at each distance window.
# If ther's a difference, paint the background in a particular color to highlight that "here wer have a statistical difference!!!"
        
### Road height ###

# Plot road height
ax2_3 = ax_3.twinx()
ax2_3.set_ylim(-10, 30)
ax2_3.grid(True, linestyle="--", linewidth=2)
ax2_3.set_ylabel('Elevation [m]', color=color_road)
road_height_plot, = ax2_3.plot(average_road_height["distance_window"], average_road_height["road_height"], 
                        color=color_road, zorder=2, linewidth=2, linestyle="--", alpha=1, label="Elevation")

# Marker for where the 110 km/h signs are
vline = ax_3.axvline(x=3100, color="k", linestyle="-", linewidth=2, label="110 km/h sign")

# Create a legend
first_patch = patches.Patch(color=color_diff_a1, label="Diff + COPE1 attempt #1")
second_patch = patches.Patch(color=color_diff, label="Diff + COPE1 attempt #2")
ax_3.legend(handles=[first_patch, second_patch, road_height_plot, vline], loc="upper right").set_zorder(2000)

ax_3.set_yticks(np.arange(0,101,10))

# Prevent secondary axis to be drawn ontop of everything
ax_3.set_zorder(1)
ax_3.patch.set_visible(False)

In [40]:
### Widgets for updating the graph ###

# Toggle highlight of both attempts
def highlightLine(x):
    id_1 = x.owner.description+"1"
    id_2 = x.owner.description+"2"
    driving_lines_obj_dict_3[id_1].highlight(x.new)
    driving_lines_obj_dict_3[id_2].highlight(x.new)

# Allow user to highlight individual attempts
checkboxes_copediff = [widgets.Label("Diff + COPE1 attempt 1 & 2:    ")]
for i in range(number_of_users):
    checkbox = widgets.Checkbox(description=str(all_user_ids[i]), value=False, layout={"width":"50px"}, indent=False)
    checkbox.observe(highlightLine, "value")
    if all_user_ids[i]%2:
        continue
    else:
        checkboxes_copediff.append(checkbox)

container_diffcope_checkboxes = widgets.HBox(checkboxes_copediff)
checkboxes = widgets.VBox([container_diffcope_checkboxes])

# Allow user to update the range of each axis
def updateXRange(x):
    margin = 200
    ax_3.set_xlim(x.new[0] - margin, x.new[1] + margin)
def updateYRange(y):
    margin = 0
    ax_3.set_ylim(y.new[0] - margin, y.new[1] + margin)
def updateYRangeSecondary(y):
    margin = 1
    ax2_3.set_ylim(y.new[0] - margin, y.new[1] + margin)
    
def createRangeSlider(min, max, step, desc):
    return widgets.IntRangeSlider(
        value=[min,max],
        min=min,
        max=max,
        step=step,
        description=desc
    )

range_slider_x = createRangeSlider(0,8000, 100, "X Range")
range_slider_x.observe(updateXRange,"value")

range_slider_y = createRangeSlider(0,140,10, "Y Range (left)")
range_slider_y.observe(updateYRange,"value")

range_slider_y_secondary = createRangeSlider(-10,40,1, "Y Range (Right)")
range_slider_y_secondary.observe(updateYRangeSecondary,"value")

# Allow user to select what to show
def toggleIndividualAttempts(show):
    global individual_attempts_are_hidden # This is needed to change global variables inside a function
    individual_attempts_are_hidden = not show.new
    for line in driving_lines_obj_dict_3:
        driving_lines_obj_dict_3[line].setHide(individual_attempts_are_hidden)
            
show_individual_attempts_checkbox = widgets.Checkbox(description="Show individual attempts", value=True)
show_individual_attempts_checkbox.observe(toggleIndividualAttempts, "value")

# TODO: Show failed attempts
def showFailedAttempts(show):
    for line in driving_lines_obj_dict_3:
        driving_lines_obj_dict_3[line].showIfFailed()
    
show_failed_attempts = widgets.Button(description="Show failed attempts")
show_failed_attempts.on_click(showFailedAttempts)
    
def saveImage(save):
    fig_3.savefig("../figures-local/a12_diff_cope_speed_distance.png", dpi=300)

save_plot = widgets.Button(description="Save as image")
save_plot.on_click(saveImage)    

# TODO: Show successful attempts

# TODO: Show average speed

def toggleMeanSTD(show):
    for plot_obj in mean_std_obj_dict_3:
        mean_std_obj_dict_3[plot_obj].setHide(not show.new)

toggle_mean_std = widgets.Checkbox(description="Show mean & std", value=True)
toggle_mean_std.observe(toggleMeanSTD, "value")

box_2 = widgets.HBox([range_slider_x, range_slider_y, range_slider_y_secondary])
box_3 = widgets.HBox([show_individual_attempts_checkbox, toggle_mean_std, show_failed_attempts, save_plot ])
box_1 = widgets.VBox([output, checkboxes])

display(widgets.VBox([box_1, box_2, box_3]))

<ipython-input-40-bf3edf2f6964>:29: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax_3.set_ylim(y.new[0] - margin, y.new[1] + margin)


### Attempt 1 & 2 - Guess-o-meter

In [43]:
#
# Class for the individual attempt line, modified for showing both attempts
class IndividualAttemptLine3:
    line_highlight = {"a": 1.0, "w": 3.0}
    line_normal = {"a": 0.4, "w": 2.0}
    marker_highlight = 1.0
    marker_normal = 0.7
    
    def __init__(self, ax, name, group):
        self.group = group
        self.driver_id = name[2]
        self.draw_order_base = self.driver_id * 3
        self.is_highlighted = False
        self.color = color_guess if name[0] == 2 else color_guess_a1
        self.color_strong = color_guess_border if name[0] == 1 else color_diff_border
        
        # Individual Attempt
        self.line = ax.plot(group["distance_window"], group["power_usage"], 
                            color=self.color, zorder=self.draw_order_base, 
                            linewidth=self.line_normal["w"], alpha=self.line_normal["a"])[0]
        
        # Marking for where the user ended up
        self.marker = ax.plot(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                              color=self.color, zorder=self.draw_order_base + 1, marker="o", markersize=25, 
                              alpha=self.marker_normal)[0]
        
        # Mark the endpoint with the user id as well
        self.text = ax.text(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                            s=self.driver_id, color="k", size="small", zorder=self.draw_order_base + 2, clip_on=True, 
                            horizontalalignment='center', verticalalignment='center')
    
    def highlight(self, value):
        self.is_highlighted = value
        
        self.line.set_alpha(self.line_highlight["a"] if self.is_highlighted else self.line_normal["a"])
        self.line.set_linewidth(self.line_highlight["w"] if self.is_highlighted else self.line_normal["w"])
        self.line.set_zorder(500 + self.driver_id * 3 + 1 if self.is_highlighted else self.driver_id * 3 + 1)
        
        # Use path effect to see overlaps more clearly
        pe1 = [
            mpe.Stroke(linewidth=5, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.line.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.marker.set_alpha(self.marker_highlight if self.is_highlighted else self.marker_normal)
        self.marker.set_zorder(500 + self.driver_id * 3 + 2 if self.is_highlighted else self.driver_id * 3 + 2)
        self.marker.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.text.set_zorder((500 + self.driver_id * 3 + 3) if self.is_highlighted else (self.driver_id * 3 + 3))
        
        # If we highlight and the rest of the lines are hidden -> show it
        if self.is_highlighted:
            self.text.set_alpha(1)
        # If we remove highlight and the rest are hidden -> hide it
        if not self.is_highlighted and individual_attempts_are_hidden:
            self.setHide(True)
    
    def setHide(self, hide, force=False):
        # If the line is alreday highlighted don't hide it! Unless fore is set to true
        if self.is_highlighted and force == False:
            return
        
        self.line.set_alpha(0 if hide else self.line_normal["a"])
        self.marker.set_alpha(0 if hide else self.marker_normal)
        self.text.set_alpha(0 if hide else 1)

    def showIfFailed(self):
        if didUserMakeIt(self.group):
            self.setHide(True, True)
        else:
            self.setHide(False)
        
class MeanWithSTD3:
    def __init__(self, ax, name, group):
        self.color = color_guess if name[0] == 2 else color_guess_a1
        self.attempt = name[0]
        self.std_alpha = 0.4
        self.color_strong = color_guess_border if name[1] == "GuessOMeter" else color_diff_border
        
        self.average_line = ax.plot(group["distance_window"], group["power_usage"]["mean"],
            color=self.color, zorder=100, linewidth=2, alpha=1)[0]
        
        # Use path effect to see overlaps more clearly
        self.pe1 = [
            mpe.Stroke(linewidth=4, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.average_line.set_path_effects(self.pe1)
        
        self.zorder = 1 if name[1] == "GuessOMeter" else 2
        self.std_band = ax.fill_between(
            group["distance_window"],
            group["power_usage"]["mean"] + group["power_usage"]["std"],
            group["power_usage"]["mean"] - group["power_usage"]["std"],
            color=self.color, alpha=self.std_alpha, zorder=self.zorder)

    def setHide(self, hide):
        self.average_line.set_alpha(0 if hide else 1)
        self.std_band.set_alpha(0 if hide else self.std_alpha)
        self.average_line.set_path_effects([] if hide else self.pe1)

In [48]:
# Plot lines
whichAttempt = 2

# Individual attempts plot
output = widgets.Output()

with output:
    fig_4, ax_4 = plt.subplots(figsize=(10, 6))
ax_4.set_xlabel("Distance [m]")
ax_4.set_ylabel("Power [kW]")
ax_4.set_title("Average power over distance traveled: attempt 1 & 2 Guess-o-meter")
ax_4.grid(True)

### Draw the plot objects ###

# Plot the individual attempts as transparent lines. At the end of each attempt, plot a dot to mark it
# This should be separated by which attempt it was!
driving_lines_obj_dict_4 = {}
average_individual_grouped = average_individual.groupby(["attempt","evis_id","user_id"])
for name,group in average_individual_grouped:
    if name[1] != "GuessOMeter": 
        continue
    
    individual_attempt = IndividualAttemptLine3(ax_4, name, group)
    # Let the key be id+attempt
    id = str(name[2]) + str(name[0])
    driving_lines_obj_dict_4[id] = individual_attempt

# Plot the average line with +- standard deviation
average_group_based_grouped = average_group_based.groupby(["attempt","evis_id"])
mean_std_obj_dict_4 = {}
for name,group in average_group_based_grouped:
    if name[1] != "GuessOMeter": 
        continue
    
    mean_with_std = MeanWithSTD3(ax_4, name, group)
    mean_std_obj_dict_4[name[0]] = mean_with_std       

sampled_distances = np.arange(0, 8000 + 1, distanceWindowWidth)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_diff = individual_pivot.round(1)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_guess = individual_pivot.round(1)

# IDEA: Check for statistical difference using ttest at each distance window.
# If ther's a difference, paint the background in a particular color to highlight that "here wer have a statistical difference!!!"
        
### Road height ###

# Plot road height
ax2_4 = ax_4.twinx()
ax2_4.set_ylim(-10, 30)
ax2_4.grid(True, linestyle="--", linewidth=2)
ax2_4.set_ylabel('Elevation [m]', color=color_road)
road_height_plot, = ax2_4.plot(average_road_height["distance_window"], average_road_height["road_height"], 
                        color=color_road, zorder=2, linewidth=2, linestyle="--", alpha=1, label="Elevation")

# Marker for where the 110 km/h signs are
vline = ax_4.axvline(x=3100, color="k", linestyle="-", linewidth=2, label="110 km/h sign")

# Create a legend
first_patch = patches.Patch(color=color_guess_a1, label="Guess-o-meter attempt #1")
second_patch = patches.Patch(color=color_guess, label="Guess-o-meter attempt #2")
ax_4.legend(handles=[first_patch, second_patch, road_height_plot, vline], loc="upper right").set_zorder(2000)

ax_4.set_yticks(np.arange(0,101,10))

# Prevent secondary axis to be drawn ontop of everything
ax_4.set_zorder(1)
ax_4.patch.set_visible(False)

In [49]:
### Widgets for updating the graph ###

# Toggle highlight of both attempts
def highlightLine(x):
    id_1 = x.owner.description+"1"
    id_2 = x.owner.description+"2"
    driving_lines_obj_dict_4[id_1].highlight(x.new)
    driving_lines_obj_dict_4[id_2].highlight(x.new)

# Allow user to highlight individual attempts
checkboxes_guessometer = [widgets.Label("Diff + COPE1 attempt 1 & 2:    ")]
for i in range(number_of_users):
    checkbox = widgets.Checkbox(description=str(all_user_ids[i]), value=False, layout={"width":"50px"}, indent=False)
    checkbox.observe(highlightLine, "value")
    if all_user_ids[i]%2:
        checkboxes_guessometer.append(checkbox)

container_diffcope_checkboxes = widgets.HBox(checkboxes_guessometer)
checkboxes = widgets.VBox([container_diffcope_checkboxes])

# Allow user to update the range of each axis
def updateXRange(x):
    margin = 200
    ax_4.set_xlim(x.new[0] - margin, x.new[1] + margin)
def updateYRange(y):
    margin = 0
    ax_4.set_ylim(y.new[0] - margin, y.new[1] + margin)
def updateYRangeSecondary(y):
    margin = 1
    ax2_4.set_ylim(y.new[0] - margin, y.new[1] + margin)
    
def createRangeSlider(min, max, step, desc):
    return widgets.IntRangeSlider(
        value=[min,max],
        min=min,
        max=max,
        step=step,
        description=desc
    )

range_slider_x = createRangeSlider(0,8000, 100, "X Range")
range_slider_x.observe(updateXRange,"value")

range_slider_y = createRangeSlider(0,140,10, "Y Range (left)")
range_slider_y.observe(updateYRange,"value")

range_slider_y_secondary = createRangeSlider(-10,40,1, "Y Range (Right)")
range_slider_y_secondary.observe(updateYRangeSecondary,"value")

# Allow user to select what to show
def toggleIndividualAttempts(show):
    global individual_attempts_are_hidden # This is needed to change global variables inside a function
    individual_attempts_are_hidden = not show.new
    for line in driving_lines_obj_dict_4:
        driving_lines_obj_dict_4[line].setHide(individual_attempts_are_hidden)
            
show_individual_attempts_checkbox = widgets.Checkbox(description="Show individual attempts", value=True)
show_individual_attempts_checkbox.observe(toggleIndividualAttempts, "value")

# TODO: Show failed attempts
def showFailedAttempts(show):
    for line in driving_lines_obj_dict_4:
        driving_lines_obj_dict_4[line].showIfFailed()
    
show_failed_attempts = widgets.Button(description="Show failed attempts")
show_failed_attempts.on_click(showFailedAttempts)
    
def saveImage(save):
    fig_4.savefig("../figures-local/a12_guess_speed_distance.png", dpi=300)

save_plot = widgets.Button(description="Save as image")
save_plot.on_click(saveImage)
    
# TODO: Show successful attempts

# TODO: Show average speed

def toggleMeanSTD(show):
    for plot_obj in mean_std_obj_dict_4:
        mean_std_obj_dict_4[plot_obj].setHide(not show.new)

toggle_mean_std = widgets.Checkbox(description="Show mean & std", value=True)
toggle_mean_std.observe(toggleMeanSTD, "value")

box_2 = widgets.HBox([range_slider_x, range_slider_y, range_slider_y_secondary])
box_3 = widgets.HBox([show_individual_attempts_checkbox, toggle_mean_std, show_failed_attempts, save_plot])
box_1 = widgets.VBox([output, checkboxes])

display(widgets.VBox([box_1, box_2, box_3]))

### Attempt 1 & 2 - Guess-o-meter with range estimate

In [50]:
#
# Class for the individual attempt line, modified for showing both attempts
class IndividualAttemptLine3:
    line_highlight = {"a": 1.0, "w": 3.0}
    line_normal = {"a": 0.4, "w": 2.0}
    marker_highlight = 1.0
    marker_normal = 0.7
    
    def __init__(self, ax, name, group):
        self.group = group
        self.driver_id = name[2]
        self.draw_order_base = self.driver_id * 3
        self.is_highlighted = False
        self.color = color_guess if name[0] == 2 else color_guess_a1
        self.color_strong = color_guess_border if name[0] == 1 else color_diff_border
        
        # Individual Attempt
        self.line = ax.plot(group["distance_window"], group["power_usage"], 
                            color=self.color, zorder=self.draw_order_base, 
                            linewidth=self.line_normal["w"], alpha=self.line_normal["a"])[0]
        
        # Marking for where the user ended up
        self.marker = ax.plot(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                              color=self.color, zorder=self.draw_order_base + 1, marker="o", markersize=25, 
                              alpha=self.marker_normal)[0]
        
        # Mark the endpoint with the user id as well
        self.text = ax.text(group["distance_window"].iloc[-1], group["power_usage"].iloc[-1], 
                            s=self.driver_id, color="k", size="small", zorder=self.draw_order_base + 2, clip_on=True, 
                            horizontalalignment='center', verticalalignment='center')
    
    def highlight(self, value):
        self.is_highlighted = value
        
        self.line.set_alpha(self.line_highlight["a"] if self.is_highlighted else self.line_normal["a"])
        self.line.set_linewidth(self.line_highlight["w"] if self.is_highlighted else self.line_normal["w"])
        self.line.set_zorder(500 + self.driver_id * 3 + 1 if self.is_highlighted else self.driver_id * 3 + 1)
        
        # Use path effect to see overlaps more clearly
        pe1 = [
            mpe.Stroke(linewidth=5, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.line.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.marker.set_alpha(self.marker_highlight if self.is_highlighted else self.marker_normal)
        self.marker.set_zorder(500 + self.driver_id * 3 + 2 if self.is_highlighted else self.driver_id * 3 + 2)
        self.marker.set_path_effects(pe1 if self.is_highlighted else [])
        
        self.text.set_zorder((500 + self.driver_id * 3 + 3) if self.is_highlighted else (self.driver_id * 3 + 3))
        
        # If we highlight and the rest of the lines are hidden -> show it
        if self.is_highlighted:
            self.text.set_alpha(1)
        # If we remove highlight and the rest are hidden -> hide it
        if not self.is_highlighted and individual_attempts_are_hidden:
            self.setHide(True)
    
    def setHide(self, hide, force=False):
        # If the line is alreday highlighted don't hide it! Unless fore is set to true
        if self.is_highlighted and force == False:
            return
        
        self.line.set_alpha(0 if hide else self.line_normal["a"])
        self.marker.set_alpha(0 if hide else self.marker_normal)
        self.text.set_alpha(0 if hide else 1)

    def showIfFailed(self):
        if didUserMakeIt(self.group):
            self.setHide(True, True)
        else:
            self.setHide(False)
        
class MeanWithSTD3:
    def __init__(self, ax, name, group):
        self.color = color_guess if name[0] == 2 else color_guess_a1
        self.attempt = name[0]
        self.std_alpha = 0.4
        self.color_strong = color_guess_border if name[1] == "GuessOMeter" else color_diff_border
        
        self.average_line = ax.plot(group["distance_window"], group["power_usage"]["mean"],
            color=self.color, zorder=100, linewidth=2, alpha=1)[0]
        
        # Use path effect to see overlaps more clearly
        self.pe1 = [
            mpe.Stroke(linewidth=4, foreground="k"),
            mpe.Stroke(foreground='white',alpha=1),
            mpe.Normal()
        ]
        self.average_line.set_path_effects(self.pe1)
        
        self.zorder = 1 if name[1] == "GuessOMeter" else 2
        self.std_band = ax.fill_between(
            group["distance_window"],
            group["power_usage"]["mean"] + group["power_usage"]["std"],
            group["power_usage"]["mean"] - group["power_usage"]["std"],
            color=self.color, alpha=self.std_alpha, zorder=self.zorder)

    def setHide(self, hide):
        self.average_line.set_alpha(0 if hide else 1)
        self.std_band.set_alpha(0 if hide else self.std_alpha)
        self.average_line.set_path_effects([] if hide else self.pe1)

In [54]:
# Plot lines
whichAttempt = 2

# Individual attempts plot
output = widgets.Output()

with output:
    fig_4, ax_4 = plt.subplots(figsize=(10, 6))
ax_4.set_xlabel("Distance [m]")
ax_4.set_ylabel("Power [kW]")
ax_4.set_title("Average power over distance traveled: attempt 1 & 2 Guess-o-meter")
ax_4.grid(True)

### Draw the plot objects ###

# Plot the individual attempts as transparent lines. At the end of each attempt, plot a dot to mark it
# This should be separated by which attempt it was!
driving_lines_obj_dict_4 = {}
average_individual_grouped = average_individual.groupby(["attempt","evis_id","user_id"])
for name,group in average_individual_grouped:
    if name[1] != "GuessOMeter": 
        continue
    
    individual_attempt = IndividualAttemptLine3(ax_4, name, group)
    # Let the key be id+attempt
    id = str(name[2]) + str(name[0])
    driving_lines_obj_dict_4[id] = individual_attempt

# Plot the average line with +- standard deviation
average_group_based_grouped = average_group_based.groupby(["attempt","evis_id"])
mean_std_obj_dict_4 = {}
for name,group in average_group_based_grouped:
    if name[1] != "GuessOMeter": 
        continue
    
    mean_with_std = MeanWithSTD3(ax_4, name, group)
    mean_std_obj_dict_4[name[0]] = mean_with_std       

sampled_distances = np.arange(0, 8000 + 1, distanceWindowWidth)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_diff = individual_pivot.round(1)

# All the averaged values for each individual
individual = average_individual[["user_id","attempt","evis_id","distance_window","power_usage"]] # Get the columns we need
individual = individual[(individual["attempt"] == whichAttempt) & (individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
individual_pivot = individual.pivot(index="user_id", columns="distance_window", values="power_usage") # Get the average speeds for each distance window for each user
a1_s1_guess = individual_pivot.round(1)

# IDEA: Check for statistical difference using ttest at each distance window.
# If ther's a difference, paint the background in a particular color to highlight that "here wer have a statistical difference!!!"
        
### Road height ###

# Plot road height
ax2_4 = ax_4.twinx()
ax2_4.set_ylim(-10, 10)
ax2_4.set_yticks(np.arange(-10,11,1))
ax2_4.set_yticks([0,0,0,0,0,0,0,0,0,0,0,1,2,3,4,5,6,7,8,9,10])
ax2_4.grid(True, linestyle="-", linewidth=1)
ax2_4.set_ylabel('Range estimate [km]', color=color_road)
#road_height_plot, = ax2_4.plot(average_road_height["distance_window"], average_road_height["road_height"], 
#                        color=color_road, zorder=2, linewidth=2, linestyle="--", alpha=1, label="Range estimate")


# TODO: Add average range estimate shown to the users!
# In Unity I'm using toString("F0") which cutts of all decimals without rounding!

average_seen_range_estimate = average_group_based[(average_group_based["evis_id"] == "GuessOMeter") & (average_group_based["attempt"] == 1)][["distance_window","range_estimate"]]
average_seen_range_estimate["seen_range_estimate"] = average_seen_range_estimate["range_estimate"]["mean"].apply(lambda x: int(x))
range_estimate_a1 = ax2_4.plot(average_seen_range_estimate["distance_window"], average_seen_range_estimate["seen_range_estimate"],
                        color=color_guess_a1, zorder=20000, linewidth=3, linestyle="--", alpha=1, label="Range estimate attempt #1")[0]

average_seen_range_estimate = average_group_based[(average_group_based["evis_id"] == "GuessOMeter") & (average_group_based["attempt"] == 2)][["distance_window","range_estimate"]]
average_seen_range_estimate["seen_range_estimate"] = average_seen_range_estimate["range_estimate"]["mean"].apply(lambda x: int(x))
range_estimate_a2 = ax2_4.plot(average_seen_range_estimate["distance_window"], average_seen_range_estimate["seen_range_estimate"],
                        color=color_guess, zorder=20000, linewidth=3, linestyle="--", alpha=1, label="Range estimate attempt #2")[0]


# Marker for where the 110 km/h signs are
vline = ax_4.axvline(x=3100, color="k", linestyle="-", linewidth=2, label="110 km/h sign")

# Create a legend
first_patch = patches.Patch(color=color_guess_a1, label="Guess-o-meter attempt #1")
second_patch = patches.Patch(color=color_guess, label="Guess-o-meter attempt #2")
ax_4.legend(handles=[first_patch, second_patch, range_estimate_a1, range_estimate_a2, vline], loc="upper right").set_zorder(2000)

ax_4.set_yticks(np.arange(0,100,10))

# Prevent secondary axis to be drawn ontop of everything
ax_4.set_zorder(1)
ax_4.patch.set_visible(False)

In [55]:
### Widgets for updating the graph ###

# Toggle highlight of both attempts
def highlightLine(x):
    id_1 = x.owner.description+"1"
    id_2 = x.owner.description+"2"
    driving_lines_obj_dict_4[id_1].highlight(x.new)
    driving_lines_obj_dict_4[id_2].highlight(x.new)

# Allow user to highlight individual attempts
checkboxes_guessometer = [widgets.Label("Diff + COPE1 attempt 1 & 2:    ")]
for i in range(number_of_users):
    checkbox = widgets.Checkbox(description=str(all_user_ids[i]), value=False, layout={"width":"50px"}, indent=False)
    checkbox.observe(highlightLine, "value")
    if all_user_ids[i]%2:
        checkboxes_guessometer.append(checkbox)

container_diffcope_checkboxes = widgets.HBox(checkboxes_guessometer)
checkboxes = widgets.VBox([container_diffcope_checkboxes])

# Allow user to update the range of each axis
def updateXRange(x):
    margin = 200
    ax_4.set_xlim(x.new[0] - margin, x.new[1] + margin)
def updateYRange(y):
    margin = 0
    ax_4.set_ylim(y.new[0] - margin, y.new[1] + margin)
def updateYRangeSecondary(y):
    margin = 1
    ax2_4.set_ylim(y.new[0] - margin, y.new[1] + margin)
    
def createRangeSlider(min, max, step, desc):
    return widgets.IntRangeSlider(
        value=[min,max],
        min=min,
        max=max,
        step=step,
        description=desc
    )

range_slider_x = createRangeSlider(0,8000, 100, "X Range")
range_slider_x.observe(updateXRange,"value")

range_slider_y = createRangeSlider(0,140,10, "Y Range (left)")
range_slider_y.observe(updateYRange,"value")

range_slider_y_secondary = createRangeSlider(-10,40,1, "Y Range (Right)")
range_slider_y_secondary.observe(updateYRangeSecondary,"value")

# Allow user to select what to show
def toggleIndividualAttempts(show):
    global individual_attempts_are_hidden # This is needed to change global variables inside a function
    individual_attempts_are_hidden = not show.new
    for line in driving_lines_obj_dict_4:
        driving_lines_obj_dict_4[line].setHide(individual_attempts_are_hidden)
            
show_individual_attempts_checkbox = widgets.Checkbox(description="Show individual attempts", value=True)
show_individual_attempts_checkbox.observe(toggleIndividualAttempts, "value")

# TODO: Show failed attempts
def showFailedAttempts(show):
    for line in driving_lines_obj_dict_4:
        driving_lines_obj_dict_4[line].showIfFailed()
    
show_failed_attempts = widgets.Button(description="Show failed attempts")
show_failed_attempts.on_click(showFailedAttempts)
    
def saveImage(save):
    fig_4.savefig("../figures-local/a12_guess_speed_distance.png", dpi=300)

save_plot = widgets.Button(description="Save as image")
save_plot.on_click(saveImage)
    
# TODO: Show successful attempts

# TODO: Show average speed

def toggleMeanSTD(show):
    for plot_obj in mean_std_obj_dict_4:
        mean_std_obj_dict_4[plot_obj].setHide(not show.new)

toggle_mean_std = widgets.Checkbox(description="Show mean & std", value=True)
toggle_mean_std.observe(toggleMeanSTD, "value")

box_2 = widgets.HBox([range_slider_x, range_slider_y, range_slider_y_secondary])
box_3 = widgets.HBox([show_individual_attempts_checkbox, toggle_mean_std, show_failed_attempts, save_plot])
box_1 = widgets.VBox([output, checkboxes])

display(widgets.VBox([box_1, box_2, box_3]))

## kW over distance

# Average speed over specific sections of the track
Answers from Research gate about ideas:
* Dave Morse:  
    Run ordinary independent t-test at each case's average speed over some specific interval **for which you have all the data** (Ask question regarding this)  
    Run ordinary independent t-test on each case's average speed for the duration of the drive  
    Equality of variance test: Calculate variance for each user at each 50m interval, and compare these between groups with a t-test (most likely will have to use robust t-method like Welch, or bootstrapped error estimate).
    Fancier method: Apply a repeated measures framework, and test whether there are any trends in speeds. And determine whether the prevalence of one trend was comparable across groups
    Completion test: use Binomial test
    
* Rainer Duesing
    Multilevel model (ML)
    

## Visualized as a bar chart with STD

S1 = [0m, 3100m]   
S2 = [3100m, 6350m]  
S3 = [6350m, 8000m]  

In [72]:
# Plot average speed over certain sections
def getSectionSpecificData(window, evis, attempt):   
    # Select all individual drives within the given distance window span, evis, and attempt nr
    return average_individual[
        (average_individual.distance_window > window[0]) & (average_individual.distance_window < window[1]) &
        (average_individual.evis_id == evis) &
        (average_individual.attempt == attempt)
    ]

sections = [[0, 3100], [3100, 6350], [6350, 8000]]

mean_std_data_frames = []
for j,section in enumerate(sections):
    diff_values = []
    guess_values = []
    # Get the values for both diff & guess at attempt 1 & 2
    for i in range(2):
        diff = getSectionSpecificData(section, "DiffAndCOPE1", i+1)
        diff = diff.describe()
        # Create a new dataframe containing only what we need
        speed_vals_diff = diff.loc[["mean","std"]][["power_usage"]] # Get the mean and std speed
        speed_vals_diff.columns = ["a" + str(i+1) + "_s" + str(j+1) + "_diff_power"]
        speed_vals_diff = speed_vals_diff.T

        guess = getSectionSpecificData(section, "GuessOMeter", i+1)
        guess = guess.describe()
        speed_vals_guess = guess.loc[["mean","std"]][["power_usage"]]
        speed_vals_guess.columns = ["a" + str(i+1) + "_s" + str(j+1) + "_guess_power"]
        speed_vals_guess = speed_vals_guess.T

        diff_values.append(speed_vals_diff)
        guess_values.append(speed_vals_guess)

    mean_std_data_frames.extend(diff_values)
    mean_std_data_frames.extend(guess_values)
        
section_data = pd.concat(mean_std_data_frames)
    
### Plot the bar chart with mean speeds over given road sections ###

# Individual attempts plot
output = widgets.Output()

with output:
    fig_bars, ax_bars = plt.subplots(figsize=(10, 6))

evis = ["COPE1 + Diff", "Guess-o-meter"]
x_pos = np.arange(len(sections))
width = 0.22
capsize=10

# Plot each section separately
for i in range(int(len(sections))):
    row = i * 4
    bar1 = ax_bars.bar(x_pos[i] - width, 
                section_data.iloc[row]["mean"], yerr=section_data.iloc[row]["std"], 
                width=width, label="Diff + COPE1 attempt #1", color=color_diff, hatch="//", edgecolor=color_diff_border, capsize=capsize)
    
    bar2 = ax_bars.bar(x_pos[i] , 
                section_data.iloc[row+1]["mean"], yerr=section_data.iloc[row+1]["std"], 
                width=width, label="Diff + COPE1 attempt #2", color=color_diff, edgecolor=color_diff_border, capsize=capsize)
    
    bar3 = ax_bars.bar(x_pos[i] + width,
                section_data.iloc[row+2]["mean"], yerr=section_data.iloc[row+2]["std"],
                width=width, label="Guess-o-meter attempt #1", color=color_guess, hatch="//", edgecolor=color_guess_border, capsize=capsize)
    
    
    bar4 = ax_bars.bar(x_pos[i] + 2 * width,
                section_data.iloc[row+3]["mean"], yerr=section_data.iloc[row+3]["std"],
                width=width, label="Guess-o-meter attempt #2", color=color_guess, edgecolor=color_guess_border, capsize=capsize)
    
plt.xticks(x_pos + width / 2, 
           ("S1="+str(sections[0]), "S2="+str(sections[1]), "S3="+str(sections[2])))

plt.yticks(np.arange(-5,51,5))

ax_bars.legend(handles=[bar1, bar2, bar3, bar4], loc="best")

ax_bars.yaxis.grid(True)
ax_bars.set_ylabel("SoC [kWh]")
ax_bars.set_xlabel("Road sections [m]")
ax_bars.set_title("Average power at section S1, S2, S3 for both EVIS during both attempts")

Text(0.5, 1.0, 'Average power at section S1, S2, S3 for both EVIS during both attempts')

In [73]:
# Widgets
def saveImage(save):
    fig_bars.savefig("../figures-local/../figures-local/road_sections_average_speed.png", dpi=300)

save_plot = widgets.Button(description="Save as image")
save_plot.on_click(saveImage)

display(widgets.VBox([output, save_plot]))

## Test statistical significance t-test
The idea here is to see if there's any statistical significance in average SoC between the two EVIS at the specified sections.  
We can also test if there's a difference between the average SoC between the attempts at sections for the two EVIS

### Create data frames with the speed for each user at each distance window
This results in 4 separate data frames. Each containing the average speed at each distance window for each user in each group
* Attempt #1 using Diff + COPE1 `a1_mean_speeds_diff`
* Attempt #1 using guess-o-meter `a1_mean_speeds_guess`
* Attempt #2 using Diff + COPE1 `a2_mean_speeds_diff`
* Attempt #2 using guess-o-meter `a2_mean_speeds_guess`

In [66]:
# Separate variables storing mean speed for each user at each distance window
which_attempt = 1

# All the averaged values for each individual
individual = average_individual[(average_individual["attempt"] == which_attempt) & (average_individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
a1_mean_speeds_diff = individual.pivot(index="distance_window", columns="user_id", values="current_soc") # Get the average speeds for each distance window for each user

# All the averaged values for each individual
individual = average_individual[(average_individual["attempt"] == which_attempt) & (average_individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
a1_mean_speeds_guess = individual.pivot(index="distance_window", columns="user_id", values="current_soc") # Create a table with just the userID and average speeds at each distance window

which_attempt = 2

# All the averaged values for each individual
individual = average_individual[(average_individual["attempt"] == which_attempt) & (average_individual["evis_id"] == "DiffAndCOPE1")] # Get one particular attempt and evis values
a2_mean_speeds_diff = individual.pivot(index="distance_window", columns="user_id", values="current_soc") # Get the average speeds for each distance window for each user

# All the averaged values for each individual
individual = average_individual[(average_individual["attempt"] == which_attempt) & (average_individual["evis_id"] == "GuessOMeter")] # Get one particular attempt and evis values
a2_mean_speeds_guess = individual.pivot(index="distance_window", columns="user_id", values="current_soc") # Create a table with just the userID and average speeds at each distance window

#a2_mean_speeds_guess # Show what one of them looks like

### Perform a t-test between the dashboards at each section. And also between attempts for the same dashboard
Important to see if the compared groups have equal variances, since it's a requirement for t-testing.  
Levene test can be used to test for equal variances. H0 -> Equal variances, if p < 0.05 the opposite is true

In [67]:
#sections = [[7000, 8000]]
sections = [[0, 3100], [3100, 6350], [6350, 8000], [7000, 8000]]

def ttest_independent(samp_1, samp_2):
    print("Sample 1 has " + str(samp_1.count()) + " entries")
    print("Sample 2 has " + str(samp_2.count()) + " entries")
    
    s,p = stats.levene(samp_1, samp_2)
    if p < 0.05:
        print("Unequal variance, can't do t-test")
    else:
        s,p = stats.ttest_ind(samp_1, samp_2)
        print("p-value: " + str(p))
        if p < 0.05:
            display(Markdown("<font color='green'>There is a statistical difference between the means</font>"))
        else:
            print("No statistical difference between the means")
    print("-"*20)

def ttest_dependent(samp_1, samp_2):
    print("Sample 1 has " + str(samp_1.count()) + " entries")
    print("Sample 2 has " + str(samp_2.count()) + " entries")
    
    s,p = stats.ttest_ind(samp_1, samp_2)
    print("p-value: " + str(p))
    if p < 0.05:
        display(Markdown("<font color='green'>There is a statistical difference between the means</font>"))
    else:
        print("No statistical difference between the means")
    print("-"*20)

    
for section in sections:
    print("section: " + str(section))
    
    # Get the mean speeds at the given section. Also remove users who'm didn't complete the given range
    a1_mean_speeds_diff_section = a1_mean_speeds_diff.loc[section[0]:section[1]].mean().dropna()
    a1_mean_speeds_guess_section = a1_mean_speeds_guess.loc[section[0]:section[1]].mean().dropna()
    
    # Get the mean speeds at the given section. Also remove users who'm didn't complete the given range
    a2_mean_speeds_diff_section = a2_mean_speeds_diff.loc[section[0]:section[1]].mean().dropna()
    a2_mean_speeds_guess_section = a2_mean_speeds_guess.loc[section[0]:section[1]].mean().dropna()
    
    # t-test for attempt #1 between the EVIS
    print("Performing independent t-test between the two EVIS for attempt #1 at road section " + str(section))
    ttest_independent(a1_mean_speeds_diff_section, a1_mean_speeds_guess_section)
    
    # t-test for attempt #2 between the EVIS
    print("Performing independent t-test between the two EVIS for attempt #2 at road section " + str(section))
    ttest_independent(a2_mean_speeds_diff_section, a2_mean_speeds_guess_section)
    
    # t-test between the two attempts for the diff + COPE1 EVIS
    print("Performing dependent t-test between the two attempts using Diff + COPE1 at road section " + str(section))
    ttest_dependent(a1_mean_speeds_diff_section, a2_mean_speeds_diff_section)
    
    # t-test between the two attempts for the guess-o-meter EVIS
    print("Performing dependent t-test between the two attempts using the guess-o-meter at road section " + str(section))
    ttest_dependent(a1_mean_speeds_guess_section, a2_mean_speeds_guess_section)
    
    print("\n")

section: [0, 3100]
Performing independent t-test between the two EVIS for attempt #1 at road section [0, 3100]
Sample 1 has 13 entries
Sample 2 has 12 entries
p-value: 0.7611109025764539
No statistical difference between the means
--------------------
Performing independent t-test between the two EVIS for attempt #2 at road section [0, 3100]
Sample 1 has 13 entries
Sample 2 has 12 entries
p-value: 0.2970977520246269
No statistical difference between the means
--------------------
Performing dependent t-test between the two attempts using Diff + COPE1 at road section [0, 3100]
Sample 1 has 13 entries
Sample 2 has 13 entries
p-value: 0.04666317429167647


<font color='green'>There is a statistical difference between the means</font>

--------------------
Performing dependent t-test between the two attempts using the guess-o-meter at road section [0, 3100]
Sample 1 has 12 entries
Sample 2 has 12 entries
p-value: 0.3657248799795928
No statistical difference between the means
--------------------


section: [3100, 6350]
Performing independent t-test between the two EVIS for attempt #1 at road section [3100, 6350]
Sample 1 has 13 entries
Sample 2 has 12 entries
p-value: 0.27302969647449504
No statistical difference between the means
--------------------
Performing independent t-test between the two EVIS for attempt #2 at road section [3100, 6350]
Sample 1 has 13 entries
Sample 2 has 12 entries
p-value: 0.4177476349831214
No statistical difference between the means
--------------------
Performing dependent t-test between the two attempts using Diff + COPE1 at road section [3100, 6350]
Sample 1 has 13 entries
Sample 2 has 13 entries
p-value: 5.1761500660763906e-05


<font color='green'>There is a statistical difference between the means</font>

--------------------
Performing dependent t-test between the two attempts using the guess-o-meter at road section [3100, 6350]
Sample 1 has 12 entries
Sample 2 has 12 entries
p-value: 0.0012140299951833226


...

--------------------


section: [6350, 8000]
Performing independent t-test between the two EVIS for attempt #1 at road section [6350, 8000]
Sample 1 has 9 entries
Sample 2 has 3 entries
p-value: 0.22211092041648414
No statistical difference between the means
--------------------
Performing independent t-test between the two EVIS for attempt #2 at road section [6350, 8000]
Sample 1 has 13 entries
Sample 2 has 11 entries
p-value: 0.8852282656067632
No statistical difference between the means
--------------------
Performing dependent t-test between the two attempts using Diff + COPE1 at road section [6350, 8000]
Sample 1 has 9 entries
Sample 2 has 13 entries
p-value: 0.003113770317745566


<font color='green'>There is a statistical difference between the means</font>

--------------------
Performing dependent t-test between the two attempts using the guess-o-meter at road section [6350, 8000]
Sample 1 has 3 entries
Sample 2 has 11 entries
p-value: 0.7106209576056651
No statistical difference between the means
--------------------


section: [7000, 8000]
Performing independent t-test between the two EVIS for attempt #1 at road section [7000, 8000]
Sample 1 has 9 entries
Sample 2 has 3 entries
p-value: 0.20550910389217797
No statistical difference between the means
--------------------
Performing independent t-test between the two EVIS for attempt #2 at road section [7000, 8000]
Sample 1 has 13 entries
Sample 2 has 10 entries
p-value: 0.6145421503675843
No statistical difference between the means
--------------------
Performing dependent t-test between the two attempts using Diff + COPE1 at road section [7000, 8000]
Sample 1 has 9 entries
Sample 2 has 13 entries
p-value: 0.006015717218690437


<font color='green'>There is a statistical difference between the means</font>

--------------------
Performing dependent t-test between the two attempts using the guess-o-meter at road section [7000, 8000]
Sample 1 has 3 entries
Sample 2 has 10 entries
p-value: 0.477571082989773
No statistical difference between the means
--------------------


